In [171]:
import pandas as pd

In [172]:
feature_dir = 'data/features/'

In [173]:
df_twitter = pd.read_csv(feature_dir+'twitter_features.csv')
df_news = pd.read_csv(feature_dir+'news_features.csv')
df_out= pd.read_csv(feature_dir+'index_features.csv')

In [177]:
df = pd.merge(df_out, df_twitter, how='left', on='Date')
df2 = pd.merge(df_out, df_news, how='left', on='Date')
df3 = pd.merge(df, df2, how='outer', on='Date')

In [175]:
df.corr()

,Change VIX,Lag 2 Change VIX,Lag 2 Significant VIX,Change SPY,Lag 2 Change SPY,Lag 2 Significant SPY,BarackObama_sentiment_score,BarackObama_heuristic_score,cnnbrk_sentiment_score,cnnbrk_heuristic_score,KimKardashian_sentiment_score,KimKardashian_heuristic_score
Change VIX,1.000000,-0.079333,-0.170544,-0.797420,-0.026668,-0.063880,0.034218,0.045193,0.012339,0.007229,-0.004164,0.002664
Lag 2 Change VIX,-0.079333,1.000000,0.756050,0.106258,-0.790251,-0.683784,-0.083132,-0.095556,0.212274,0.212178,0.022989,0.018752
Lag 2 Significant VIX,-0.170544,0.756050,1.000000,0.146967,-0.577632,-0.506726,-0.006107,-0.015632,0.213348,0.229125,0.067904,0.053826
Change SPY,-0.797420,0.106258,0.146967,1.000000,-0.054867,0.001392,-0.030472,-0.034342,0.014805,0.014589,-0.031171,-0.030169
Lag 2 Change SPY,-0.026668,-0.790251,-0.577632,-0.054867,1.000000,0.819734,0.092629,0.099958,-0.150117,-0.133908,-0.024085,-0.017504
Lag 2 Significant SPY,-0.063880,-0.683784,-0.506726,0.001392,0.819734,1.000000,0.102171,0.105238,-0.043143,-0.031752,-0.065448,-0.070359
BarackObama_sentiment_score,0.034218,-0.083132,-0.006107,-0.030472,0.092629,0.102171,1.000000,0.995079,-0.208320,-0.209919,0.007825,0.008337
BarackObama_heuristic_score,0.045193,-0.095556,-0.015632,-0.034342,0.099958,0.105238,0.995079,1.000000,-0.203924,-0.204932,0.008332,0.008980
cnnbrk_sentiment_score,0.012339,0.212274,0.213348,0.014805,-0.150117,-0.043143,-0.208320,-0.203924,1.000000,0.993250,0.204174,0.196099
cnnbrk_heuristic_score,0.007229,0.212178,0.229125,0.014589,-0.133908,-0.031752,-0.209919,-0.204932,0.993250,1.000000,0.203651,0.194903


In [176]:
df2.corr()

,Change VIX,Lag 2 Change VIX,Lag 2 Significant VIX,Change SPY,Lag 2 Change SPY,Lag 2 Significant SPY,New York Times_sentiment_score,CNN_sentiment_score,Washington Post_sentiment_score,NYT_sentiment_score
Change VIX,1.000000,-0.160492,-0.185877,-0.765581,-0.052135,-0.105625,-0.013360,0.039379,0.094973,0.163898
Lag 2 Change VIX,-0.160492,1.000000,0.762357,0.080632,-0.697852,-0.499780,-0.028221,0.095268,0.029537,0.090709
Lag 2 Significant VIX,-0.185877,0.762357,1.000000,0.097820,-0.539609,-0.372859,-0.020057,0.062206,-0.026315,0.061262
Change SPY,-0.765581,0.080632,0.097820,1.000000,0.040681,0.137014,0.008744,-0.091346,-0.126565,-0.089267
Lag 2 Change SPY,-0.052135,-0.697852,-0.539609,0.040681,1.000000,0.802537,0.108118,0.003429,0.035303,0.004922
Lag 2 Significant SPY,-0.105625,-0.499780,-0.372859,0.137014,0.802537,1.000000,0.090930,-0.032982,-0.029916,-0.029337
New York Times_sentiment_score,-0.013360,-0.028221,-0.020057,0.008744,0.108118,0.090930,1.000000,0.009178,0.033991,0.056291
CNN_sentiment_score,0.039379,0.095268,0.062206,-0.091346,0.003429,-0.032982,0.009178,1.000000,0.125188,0.127396
Washington Post_sentiment_score,0.094973,0.029537,-0.026315,-0.126565,0.035303,-0.029916,0.033991,0.125188,1.000000,0.110816
NYT_sentiment_score,0.163898,0.090709,0.061262,-0.089267,0.004922,-0.029337,0.056291,0.127396,0.110816,1.000000


In [178]:
df3

,Date,Change VIX_x,Lag 2 Change VIX_x,Lag 2 Significant VIX_x,Change SPY_x,Lag 2 Change SPY_x,Lag 2 Significant SPY_x,BarackObama_sentiment_score,BarackObama_heuristic_score,cnnbrk_sentiment_score,...,Change VIX_y,Lag 2 Change VIX_y,Lag 2 Significant VIX_y,Change SPY_y,Lag 2 Change SPY_y,Lag 2 Significant SPY_y,New York Times_sentiment_score,CNN_sentiment_score,Washington Post_sentiment_score,NYT_sentiment_score
0,2015-01-02,0.116279,0.144707,1,-0.007741,-0.020787,-1,-0.4289,-3.145348,NaN,...,0.116279,0.144707,1,-0.007741,-0.020787,-1,NaN,-4.2102,NaN,-8.5101
1,2015-01-02,0.116279,0.144707,1,-0.007741,-0.020787,-1,-0.4289,-3.145348,NaN,...,0.116279,0.144707,1,-0.007741,-0.020787,-1,NaN,-1.7360,NaN,-8.5101
2,2015-01-02,0.116279,0.144707,1,-0.007741,-0.020787,-1,-0.4289,-3.145348,NaN,...,0.116279,0.144707,1,-0.007741,-0.020787,-1,NaN,-0.4019,NaN,-8.5101
3,2015-01-02,0.116279,0.144707,1,-0.007741,-0.020787,-1,-0.4289,-3.145348,NaN,...,0.116279,0.144707,1,-0.007741,-0.020787,-1,NaN,-4.2102,NaN,-8.5101
4,2015-01-02,0.116279,0.144707,1,-0.007741,-0.020787,-1,-0.4289,-3.145348,NaN,...,0.116279,0.144707,1,-0.007741,-0.020787,-1,NaN,-1.7360,NaN,-8.5101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58738,2016-12-30,0.003802,-0.031818,-1,0.001114,0.003960,0,NaN,NaN,-2.0089,...,0.003802,-0.031818,-1,0.001114,0.003960,0,-0.495,-1.1880,0.9239,-6.2017
58739,2016-12-30,0.003802,-0.031818,-1,0.001114,0.003960,0,NaN,NaN,-2.0089,...,0.003802,-0.031818,-1,0.001114,0.003960,0,-0.495,-1.1880,-1.7894,-6.2017
58740,2016-12-30,0.003802,-0.031818,-1,0.001114,0.003960,0,NaN,NaN,-2.0089,...,0.003802,-0.031818,-1,0.001114,0.003960,0,-0.495,-0.6568,0.6944,-6.2017
58741,2016-12-30,0.003802,-0.031818,-1,0.001114,0.003960,0,NaN,NaN,-2.0089,...,0.003802,-0.031818,-1,0.001114,0.003960,0,-0.495,-0.6568,0.9239,-6.2017
